In [3]:
from qiskit import IBMQ

IBMQ.save_account("4a2b1a36805ecf250f886c317cafd3bbdf55633282b8df8b8b08857d20356615132b9d174cc3a31bd54ccabf228f08658ebedf07f8b52c67619bdf0dff703bc7")
provider = IBMQ.load_account()

SyntaxError: positional argument follows keyword argument (1641748963.py, line 3)

In [9]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

In [16]:
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import TrainableFidelityQuantumKernel
from qiskit_machine_learning.kernels.algorithms import QuantumKernelTrainer
from qiskit.circuit.library import ZZFeatureMap

In [31]:
from qiskit_machine_learning.kernels import QuantumKernel


ImportError: cannot import name 'QuantumKernel' from 'qiskit_machine_learning.kernels' (/Users/lukepower/opt/anaconda3/lib/python3.9/site-packages/qiskit_machine_learning/kernels/__init__.py)

In [17]:
import numpy as np
import matplotlib.pyplot as plt

In [29]:
class QSVM_routine:
    def __init__(self, feature_dim=2, train_test_split=0.3, train_samples=5, test_samples=2, seed=0, shots=1024):
        self.feature_dim = feature_dim
        self.train_test_split = train_test_split
        self.train_samples = train_samples
        self.test_samples = test_samples
        self.seed = seed
        self.shots = shots

    def train_test_datasets(self):
        self.class_labels = [r'A', r'B']
        data, target = datasets.load_breast_cancer(return_X_y=True)
        train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=self.train_test_split, random_state=self.seed)
        
        # Normalization
        scaler = StandardScaler().fit(train_X)
        train_X = scaler.transform(train_X)
        test_X = scaler.transform(test_X)
        
        # PCA for dimensionality reduction
        pca = PCA(n_components=self.feature_dim).fit(train_X)
        train_X = pca.transform(train_X)
        test_X = pca.transform(test_X)
        
        # Scaling to (-1, +1)
        min_max_scaler = MinMaxScaler((-1, 1)).fit(np.concatenate((train_X, test_X), axis=0))
        train_X = min_max_scaler.transform(train_X)
        test_X = min_max_scaler.transform(test_X)
        
        # Selecting samples
        self.train_X = train_X[:self.train_samples]
        self.train_y = train_y[:self.train_samples]
        self.test_X = test_X[:self.test_samples]
        self.test_y = test_y[:self.test_samples]

    def train_model(self):
        backend = Aer.get_backend('qasm_simulator')
        feature_map = ZZFeatureMap(feature_dimension=self.feature_dim, reps=2, entanglement='linear')
        quantum_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=QuantumInstance(backend, shots=self.shots))
        
        qsvc = QSVC(quantum_kernel=quantum_kernel)
        qsvc.fit(self.train_X, self.train_y)
        score = qsvc.score(self.test_X, self.test_y)
        
        print(f"Test Accuracy: {score}")
        
    def main(self):
        self.train_test_datasets()
        self.train_model()

In [30]:
if __name__ == '__main__':
    qsvm_routine = QSVM_routine()
    qsvm_routine.main()

NameError: name 'QuantumKernel' is not defined